In [ ]:
!pip -q install "transformers==4.55.0" "tokenizers==0.21.4" "pandas==2.2.2" "eth-hash[pycryptodome]==0.5.2" "requests==2.32.3"


In [ ]:
import os, gzip, shutil, json, time, zipfile
from pathlib import Path
import requests, pandas as pd
from eth_hash.auto import keccak
from transformers import AutoTokenizer

# Config (match your paper)
ZIP_URL   = "https://physionet.org/static/published-projects/mimic-iv-demo/mimic-iv-clinical-database-demo-2.2.zip"
ROW_LIMIT = 200
MODEL_REPO = "Qwen/Qwen2.5-3B-Instruct"   # tokenizer only (no model)
TARGET_FILE = "tokenized_admissions.jsonl"  # change if you want a different proof target

RAW_DIR   = Path("raw_data")
CANON_DIR = Path("canonical_data")
TOK_DIR   = Path("tokenized_data")
RAW_DIR.mkdir(exist_ok=True); CANON_DIR.mkdir(exist_ok=True); TOK_DIR.mkdir(exist_ok=True)


In [ ]:
from eth_hash.auto import keccak

# --- Download ZIP once ---
zip_path = Path("mimic_demo.zip")
if not zip_path.exists():
    r = requests.get(ZIP_URL, stream=True); r.raise_for_status()
    with open(zip_path, "wb") as f:
        for chunk in r.iter_content(8192): f.write(chunk)

# --- Extract ZIP ---
if not any(RAW_DIR.iterdir()):
    with zipfile.ZipFile(zip_path, "r") as zf:
        zf.extractall(RAW_DIR)

# --- Canonicalize helpers (match paper) ---
def canonicalize_csv(in_path, out_path):
    try:
        df = pd.read_csv(in_path, dtype=str)
    except Exception as e:
        print("skip:", in_path.name, e); return False
    cols = sorted(df.columns)
    df = df[cols].sort_values(by=cols, na_position="last").reset_index(drop=True)
    df.to_csv(out_path, index=False, encoding="utf-8", lineterminator="\n")
    return True

# --- Decompress .csv.gz then canonicalize all CSVs (except demo_subject_id) ---
for fp in sorted(RAW_DIR.rglob("*")):
    if not fp.is_file():
        continue
    if "demo_subject_id" in fp.name.lower():
        continue
    if fp.suffix == ".gz" and fp.name.endswith(".csv.gz"):
        dec = fp.with_suffix("")  # drop .gz
        with gzip.open(fp, "rb") as fin, open(dec, "wb") as fout:
            shutil.copyfileobj(fin, fout)
        fp = dec
    if fp.suffix.lower() == ".csv":
        out = CANON_DIR / fp.name
        if canonicalize_csv(fp, out):
            print("canonicalized:", fp.name)

# --- Tokenize (Qwen tokenizer) ---
tok = AutoTokenizer.from_pretrained(MODEL_REPO)  # uses HF hub
def build_records(csv_path):
    df = pd.read_csv(csv_path, dtype=str).head(ROW_LIMIT)
    prompts, responses = [], []
    for _, row in df.iterrows():
        # Python dict repr (single quotes; NaN -> nan)
        prompts.append(
            f"You are a clinical assistant. Given the following record from {csv_path.name}, "
            f"provide a short summary:\n{row.to_dict()}"
        )
        responses.append("Summary: [Your summary here]")
    enc = tok(prompts, text_pair=responses, truncation=True, max_length=512)
    out_path = TOK_DIR / f"tokenized_{csv_path.stem}.jsonl"
    with open(out_path, "w", encoding="utf-8") as f:
        for i in range(len(prompts)):
            rec = {
                "prompt": prompts[i],
                "response": responses[i],
                "input_ids": list(map(int, enc["input_ids"][i])),
                "attention_mask": list(map(int, enc["attention_mask"][i])),
            }
            f.write(json.dumps(rec, ensure_ascii=False) + "\n")
    return out_path

TOK_DIR.mkdir(exist_ok=True)
for csv in sorted(CANON_DIR.glob("*.csv")):
    out = build_records(csv); print("tokenized ->", out.name)

# --- Keccak(file bytes) for each JSONL ---
def file_keccak(p: Path):
    with open(p, "rb") as f:
        return keccak(f.read())

files  = sorted([p for p in TOK_DIR.iterdir() if p.suffix == ".jsonl"], key=lambda p: p.name)
leaves = [file_keccak(p) for p in files]

# === Merkle (SORTED-PAIR rule; duplicate last when odd) ===
def _parent(a: bytes, b: bytes) -> bytes:
    # match Solidity: if (a < b) hash(a||b) else hash(b||a)
    return keccak(a + b) if a < b else keccak(b + a)

def merkle_root(hashes: list[bytes]) -> bytes:
    level = hashes[:]
    if not level:
        raise ValueError("no leaves")
    while len(level) > 1:
        if len(level) % 2 == 1:
            level.append(level[-1])
        nxt = []
        for i in range(0, len(level), 2):
            nxt.append(_parent(level[i], level[i+1]))
        level = nxt
    return level[0]

def merkle_proof(hashes: list[bytes], idx: int) -> list[str]:
    proof, level_i, level = [], idx, hashes[:]
    if not (0 <= idx < len(level)):
        raise IndexError("target index out of range")
    while len(level) > 1:
        if len(level) % 2 == 1:
            level.append(level[-1])
        nxt = []
        for i in range(0, len(level), 2):
            L, R = level[i], level[i+1]
            if i == level_i or i+1 == level_i:
                sib = R if i == level_i else L
                proof.append("0x" + sib.hex())     # sibling only; no direction needed
                level_i = len(nxt)
            nxt.append(_parent(L, R))
        level = nxt
    return proof

# --- Root / proof for TARGET_FILE ---
root_bytes = merkle_root(leaves)
root       = "0x" + root_bytes.hex()

tgt_i      = next(i for i, f in enumerate(files) if f.name == TARGET_FILE)
proof      = merkle_proof(leaves, tgt_i)
leaf_hex   = "0x" + leaves[tgt_i].hex()

# --- Write artifacts for the paper & registration ---
with open("merkle_hashes.txt", "w") as f:
    f.write(f"Merkle Root: {root}\n\n")
    for p, h in zip(files, leaves):
        f.write(f"{p.name}: 0x{h.hex()}\n")
    f.write("\nProof for " + TARGET_FILE + ":\n")
    f.write("Leaf Hash: " + leaf_hex + "\n")
    f.write("Proof Array: " + str(proof) + "\n")

# Build metadata JSON for the dapp to upload to IPFS
import platform, time, json
# DATASET_NAME already defined elsewhere in your notebook; if not, set it:
DATASET_NAME = "MIMIC-IV-Demo-v2.2"
dataset_id_hex = "0x" + keccak(DATASET_NAME.encode("utf-8")).hex()

# Use the leaf hashes we already computed
file_hash_map = {p.name: "0x" + h.hex() for p, h in zip(files, leaves)}
total_size = sum(p.stat().st_size for p in files)
files_info = [{
    "filename": p.name,
    "size_bytes": p.stat().st_size,
    "keccak256": file_hash_map[p.name],
} for p in files]

preprocessing_metadata = {
    "dataset": {
        "name": DATASET_NAME,
        "dataset_id": dataset_id_hex,
        "description": "Tokenized clinical dataset prepared for Qwen2.5-3B-Instruct fine-tuning.",
        "merkle_root": root,
        "num_files": len(files_info),
        "total_size_bytes": total_size,
        "files": files_info,
        "created_at": time.strftime("%Y-%m-%dT%H:%M:%SZ", time.gmtime())
    },
    "tokenizer": { "name": MODEL_REPO },
    "prompt_template": {
        "description": "You are a clinical assistant. Given the following record from {filename}, provide a short summary.",
        "max_length": 512,
        "truncation": True
    },
    "environment": {
        "os": platform.system(),
        "python_version": platform.python_version(),
        "pandas_version": pd.__version__
    }
}

with open("preprocessing_metadata.json", "w", encoding="utf-8") as f:
    json.dump(preprocessing_metadata, f, indent=2)

# Optional: write a viewer-friendly verify bundle (no version yet)
verify_bundle = {
    "datasetId": dataset_id_hex,
    "merkleRoot": root,
    "targetFile": TARGET_FILE,
    "leafHash": leaf_hex,
    "proof": proof
}
with open("verify_bundle.json", "w", encoding="utf-8") as f:
    json.dump(verify_bundle, f, indent=2)

print("\n=== RESULTS ===")
print("Root:", root)
print("Target leaf:", leaf_hex)
print("Proof:", proof)
print("\nWrote: merkle_hashes.txt, preprocessing_metadata.json, verify_bundle.json, tokenized_data/")


In [ ]:
# === Create a lightweight verify bundle for the dapp ===
import json, time

DS_VERSION = 1  # <-- set this to the version you want to verify

verify_bundle = {
    "datasetId": dataset_id_hex,   # 0x… bytes32
    "version": DS_VERSION,         # uint256
    "leafHash": leaf_hex,          # 0x… bytes32 (target file’s keccak256)
    "proof": proof,                # ["0x…", "0x…", ...]
    "merkleRoot": root,            # optional, for display/debug
    "targetFile": TARGET_FILE,     # optional
    "timestamp": time.strftime("%Y-%m-%dT%H:%M:%SZ", time.gmtime())
}
with open("verify_bundle.json","w") as f:
    json.dump(verify_bundle, f, indent=2)

print("Wrote verify_bundle.json")
